# Google Colab Version: [Open this notebook in Google Colab](https://colab.research.google.com/github/starfishdata/starfish/blob/main/examples/data_factory.ipynb)

#### Dependencies 

In [23]:
#%pip install starfish-core
%pip install --index-url https://test.pypi.org/simple/ \
                   --extra-index-url https://pypi.org/simple \
                    starfish-core

Looking in indexes: https://test.pypi.org/simple/, https://pypi.org/simple

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [24]:
## Fix for Jupyter Notebook only — do NOT use in production
## Enables async code execution in notebooks, but may cause issues with sync/async issues
## For production, please run in standard .py files without this workaround
## See: https://github.com/erdewit/nest_asyncio for more details
import nest_asyncio
nest_asyncio.apply()

from starfish import StructuredLLM, data_factory
from starfish.llm.utils import merge_structured_outputs

from starfish.common.env_loader import load_env_file ## Load environment variables from .env file
load_env_file()

2025-05-23 22:50:10 | WARNING  | Failed to load environment variables from /Users/john/Documents/projects/aa/python/starfish/starfish/.env


In [25]:
## Helper function mock llm call
# When developing data pipelines with LLMs, making thousands of real API calls
# can be expensive. Using mock LLM calls lets you test your pipeline's reliability,
# failure handling, and recovery without spending money on API calls.
from starfish.data_factory.utils.mock import mock_llm_call

#### 3. Working with Different Input Formats


Data Factory is flexible with how you provide inputs. Let's demonstrate different ways to pass parameters to data_factory functions.

'data' is a reserved keyword expecting list(dict) or tuple(dict) - this design make it super easy to pass large data and support HuggingFace and Pandas dataframe very easily

In [26]:
## We will be using mock llm call for rest of example to save on token
## Mock LLM call is a function that simulates an LLM API call with random delays (controlled by sleep_time) and occasional failures (controlled by fail_rate)
await mock_llm_call(city_name="New York", num_records_per_city=3)

[{'answer': 'New York_3'}, {'answer': 'New York_1'}, {'answer': 'New York_5'}]

In [27]:
@data_factory(max_concurrency=100)
async def input_format_mock_llm(city_name: str, num_records_per_city: int):
    return await mock_llm_call(city_name=city_name, num_records_per_city=num_records_per_city, fail_rate=0.01)

In [28]:
# Format 1: Multiple lists that get zipped together
input_format_data1 = input_format_mock_llm.run(city_name=["New York", "London", "Tokyo", "Paris", "Sydney"], num_records_per_city=[2, 1, 1, 1, 1])

2025-05-23 22:50:10 | INFO     | [JOB START] Master Job ID: 4da82fc7-4112-4e05-b58c-53cf470747ad | Logging progress every 3 seconds
2025-05-23 22:50:10 | INFO     | [JOB PROGRESS] Completed: 0/5 | Running: 5 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:11 | INFO     | [JOB FINISHED] Final Status: Completed: 5/5 | Attempted: 5 (Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


In [29]:
# Format 2: List + single value (single value gets broadcasted)
input_format_data2 = input_format_mock_llm.run(city_name=["New York", "London", "Tokyo", "Paris", "Sydney"], num_records_per_city=1)

2025-05-23 22:50:11 | INFO     | [JOB START] Master Job ID: 73973449-6069-485e-ac8c-b1b3a6b3f1a4 | Logging progress every 3 seconds
2025-05-23 22:50:11 | INFO     | [JOB PROGRESS] Completed: 0/5 | Running: 5 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:12 | INFO     | [JOB FINISHED] Final Status: Completed: 5/5 | Attempted: 5 (Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


In [30]:
# Format 3: Special 'data' parameter
# 'data' is a reserved keyword expecting list(dict) or tuple(dict)
# Makes integration with various data sources easier
input_format_data3 = input_format_mock_llm.run(data=[{"city_name": "New York", "num_records_per_city": 2}, {"city_name": "London", "num_records_per_city": 1}, {"city_name": "Tokyo", "num_records_per_city": 1}, {"city_name": "Paris", "num_records_per_city": 1}, {"city_name": "Sydney", "num_records_per_city": 1}])

2025-05-23 22:50:12 | INFO     | [JOB START] Master Job ID: aa9954f9-fc18-4b42-959e-fb2a897987c7 | Logging progress every 3 seconds
2025-05-23 22:50:12 | INFO     | [JOB PROGRESS] Completed: 0/5 | Running: 5 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:13 | INFO     | [JOB FINISHED] Final Status: Completed: 5/5 | Attempted: 5 (Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


#### 4. Resilient error retry
Data Factory automatically handles errors and retries, making your pipelines robust.

Let's demonstrate with a high failure rate example.

In [31]:
@data_factory(max_concurrency=100)
async def high_error_rate_mock_llm(city_name: str, num_records_per_city: int):
    return await mock_llm_call(city_name=city_name, num_records_per_city=num_records_per_city, fail_rate=0.3) # Hardcode to 30% chance of failure

# Process all cities - some will fail, but data_factory keeps going
cities = ["New York", "London", "Tokyo", "Paris", "Sydney"] * 5  # 25 cities
high_error_rate_mock_lllm_data = high_error_rate_mock_llm.run(city_name=cities, num_records_per_city=1)

print(f"\nSuccessfully completed {len(high_error_rate_mock_lllm_data)} out of {len(cities)} tasks")
print("Data Factory automatically handled the failures and continued processing")
print("The results only include successful tasks")

2025-05-23 22:50:13 | INFO     | [JOB START] Master Job ID: 730b766d-3c23-419a-a3dd-271d683818b1 | Logging progress every 3 seconds
2025-05-23 22:50:13 | INFO     | [JOB PROGRESS] Completed: 0/25 | Running: 25 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:15 | ERROR    | Error running task: Mock LLM failed to process city: Tokyo
2025-05-23 22:50:15 | ERROR    | Error running task: Mock LLM failed to process city: New York
2025-05-23 22:50:16 | INFO     | [JOB PROGRESS] Completed: 23/25 | Running: 0 | Attempted: 25    (Completed: 23, Failed: 2, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:19 | INFO     | [JOB FINISHED] Final Status: Completed: 25/25 | Attempted: 27 (Failed: 2, Filtered: 0, Duplicate: 0, InDeadQueue: 0)

Successfully completed 25 out of 25 tasks
Data Factory automatically handled the failures and continued processing
The results only include successful tasks


#### 5. Resume

This is essential for long-running jobs with thousands of tasks.

If a job is interrupted, you can pick up where you left off using one of two resume methods:


1. **Same Session Resume**: If you're still in the same session where the job was interrupted, simply call - Same instance with .resume()

2. **Cross-Session Resume**: If you've closed your notebook or lost your session, you can resume using the job ID:
   ```python
   from starfish import DataFactory
   # Resume using the master job ID from a previous run
   data_factory = DataFactory.resume_from_checkpoint(job_id="your_job_id")
   ```

The key difference:
- `resume()` uses the same DataFactory instance you defined
- `resume_from_checkpoint()` reconstructs your DataFactory from persistent storage where tasks and progress are saved

> **Note**: Google Colab users may experience issues with `resume_from_checkpoint()` due to how Colab works

We're simulating an interruption here. In a real scenario, this might happen if your notebook errors out, is manually interrupted with a keyboard command, encounters API rate limits, or experiences any other issues that halt execution.

In [32]:
@data_factory(max_concurrency=10)
async def re_run_mock_llm(city_name: str, num_records_per_city: int):
    return await mock_llm_call(city_name=city_name, num_records_per_city=num_records_per_city, fail_rate=0.3)

cities = ["New York", "London", "Tokyo", "Paris", "Sydney"] * 20  # 100 cities
re_run_mock_llm_data_1 = re_run_mock_llm.run(city_name=cities, num_records_per_city=1)

2025-05-23 22:50:19 | INFO     | [JOB START] Master Job ID: 6829de29-0b83-4a64-835b-cc79cbad5e3a | Logging progress every 3 seconds
2025-05-23 22:50:19 | INFO     | [JOB PROGRESS] Completed: 0/100 | Running: 10 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:21 | ERROR    | Error running task: Mock LLM failed to process city: Paris
2025-05-23 22:50:21 | ERROR    | Error running task: Mock LLM failed to process city: Sydney
2025-05-23 22:50:21 | ERROR    | Error running task: Mock LLM failed to process city: New York
2025-05-23 22:50:21 | ERROR    | consecutive_not_completed: in 3 times, stopping this job; please adjust factory config and input data then resume_from_checkpoint(6829de29-0b83-4a64-835b-cc79cbad5e3a)
2025-05-23 22:50:21 | INFO     | [JOB FINISHED] Final Status: Completed: 17/100 | Attempted: 20 (Failed: 3, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


In [33]:
print("When a job is interrupted, you'll see a message like:")
print("[RESUME INFO] 🚨 Job stopped unexpectedly. You can resume the job by calling .resume()")

print("\nTo resume an interrupted job, simply call:")
print("interrupted_job_mock_llm.resume()")
print('')
print(f"For this example we have {len(re_run_mock_llm_data_1)}/{len(cities)} data generated and not finished yet!")

When a job is interrupted, you'll see a message like:
[RESUME INFO] 🚨 Job stopped unexpectedly. You can resume the job by calling .resume()

To resume an interrupted job, simply call:
interrupted_job_mock_llm.resume()

For this example we have 17/100 data generated and not finished yet!


In [34]:
## Lets keep continue the rest of run by resume_from_checkpoint 
re_run_mock_llm_data_2 = re_run_mock_llm.resume()

2025-05-23 22:50:22 | INFO     | [JOB RESUME START] PICKING UP FROM WHERE THE JOB WAS LEFT OFF...

2025-05-23 22:50:22 | INFO     | [RESUME PROGRESS] STATUS AT THE TIME OF RESUME: Completed: 17 / 100 | Failed: 3 | Duplicate: 0 | Filtered: 0
2025-05-23 22:50:22 | INFO     | [JOB PROGRESS] Completed: 17/100 | Running: 10 | Attempted: 20    (Completed: 17, Failed: 3, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:24 | ERROR    | Error running task: Mock LLM failed to process city: Paris
2025-05-23 22:50:24 | ERROR    | consecutive_not_completed: in 3 times, stopping this job; please adjust factory config and input data then resume_from_checkpoint(6829de29-0b83-4a64-835b-cc79cbad5e3a)
2025-05-23 22:50:24 | INFO     | [JOB FINISHED] Final Status: Completed: 30/100 | Attempted: 34 (Failed: 4, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


In [35]:
print(f"Now we still able to finished with what is left!! {len(re_run_mock_llm_data_2)} data generated!")

Now we still able to finished with what is left!! 30 data generated!


#### 6. Dry run
Before running a large job, you can do a "dry run" to test your pipeline. This only processes a single item and doesn't save state to the database.

In [36]:
@data_factory(max_concurrency=10)
async def dry_run_mock_llm(city_name: str, num_records_per_city: int):
    return await mock_llm_call(city_name=city_name, num_records_per_city=num_records_per_city, fail_rate=0.3)

dry_run_mock_llm_data = dry_run_mock_llm.dry_run(city_name=["New York", "London", "Tokyo", "Paris", "Sydney"]*20, num_records_per_city=1)

2025-05-23 22:50:24 | INFO     | [JOB START] Master Job ID: None | Logging progress every 3 seconds
2025-05-23 22:50:24 | INFO     | [JOB PROGRESS] Completed: 0/1 | Running: 1 | Attempted: 0    (Completed: 0, Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)
2025-05-23 22:50:25 | INFO     | [JOB FINISHED] Final Status: Completed: 1/0 | Attempted: 1 (Failed: 0, Filtered: 0, Duplicate: 0, InDeadQueue: 0)


#### 8. Advanced Usage
Data Factory offers more advanced capabilities for complete pipeline customization, including hooks that execute at key stages and shareable state to coordinate between tasks. These powerful features enable complex workflows and fine-grained control. Our dedicated examples for advanced data_factory usage will be coming soon!